In [ ]:
import asyncio
import requests
import pymongo
from pymongo import MongoClient
from bson.json_util import dumps
from indy import anoncreds, did, ledger, pool, wallet, IndyError

import json

async def run():
    print("Send credential offer to device -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    client = MongoClient(host='supplier_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["device1_db"]

    supplier = db.supplier_tb.find_one({"name": "Supplier"})

 
 
    supplier['IoTDevice_cred_values'] = json.dumps({
        "first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
        "last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
        "degree": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
        "status": {"raw": "graduated", "encoded": "2213454313412354"},
        "ssn": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
        "year": {"raw": "2015", "encoded": "2015"},
        "average": {"raw": "5", "encoded": "5"}
    })

    print("\"Supplier\" -> Create \"Suppplier Certificate\" Credential for IoT Device")

    supplier['transcript_cred'], _, _ = \
        await anoncreds.issuer_create_credential(supplier['wallet'], supplier['transcript_cred_offer'],
                                                 supplier['transcript_cred_request'],
                                                 supplier['IoTDevice_cred_values'], None, None)

    print("\"Supplier\" -> Send \"Suppplier Certificate\" Credential to IoT Device")